### References:
[1] Maciejewski, Filip B., Jacob Biamonte, Stuart Hadfield, and Davide Venturelli. "[Improving quantum approximate optimization by noise-directed adaptive remapping.](https://arxiv.org/abs/2404.01412)" arXiv preprint arXiv:2404.01412 (2024).

[2] Maciejewski, Filip B., Bao G. Bach, Maxime Dupont, P. Aaron Lott, Bhuvanesh Sundar, David E. Bernal Neira, Ilya Safro, and Davide Venturelli. "[A multilevel approach for solving large-scale qubo problems with noisy hybrid quantum approximate optimization.](https://arxiv.org/abs/2408.07793)" In 2024 IEEE High Performance Extreme Computing Conference (HPEC), pp. 1-10. IEEE, 2024.

[3] Maciejewski, Filip B., Stuart Hadfield, Benjamin Hall, Mark Hodson, Maxime Dupont, Bram Evert, James Sud et al. "[Design and execution of quantum circuits using tens of superconducting qubits and thousands of gates for dense Ising optimization problems.](https://arxiv.org/abs/2308.12423)" Physical Review Applied 22, no. 4 (2024): 044074.

[4] Tam, Wai-Hong, Hiromichi Matsuyama, Ryo Sakai, and Yu Yamashiro. "[Enhancing NDAR with Delay-Gate-Induced Amplitude Damping.]"(https://arxiv.org/abs/2504.12628) arXiv preprint arXiv:2504.12628 (2025).

[5] Lykov, Danylo, Ruslan Shaydulin, Yue Sun, Yuri Alexeev, and Marco Pistoia. "[Fast simulation of high-depth qaoa circuits.](https://arxiv.org/abs/2309.04841)" In Proceedings of the SC'23 Workshops of The International Conference on High Performance Computing, Network, Storage, and Analysis, pp. 1443-1451. 2023.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import os

import numpy as np

from quapopt import ancillary_functions as anf

os.makedirs('../temp', exist_ok=True)

### Generate a random Hamiltonian instance

* Like in previous notebooks, we generate a random Hamiltonian instance.

In [5]:
from quapopt.hamiltonians.generators import build_hamiltonian_generator
from quapopt.data_analysis.data_handling import (CoefficientsType,
                                                 CoefficientsDistribution,
                                                 CoefficientsDistributionSpecifier,
                                                 HamiltonianModels)
from quapopt.data_analysis.data_handling import LoggingLevel

#more qubits so we observe some effects
number_of_qubits = 21
seed_cost_hamiltonian = 1

coefficients_type = CoefficientsType.DISCRETE
coefficients_distribution = CoefficientsDistribution.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}
coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

# We generate a Hamiltonian instance. In this case it's a random Sherrington-Kirkpatrick Hamiltonian
hamiltonian_model = HamiltonianModels.SherringtonKirkpatrick
localities = (2,)
generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)

cost_hamiltonian = generator_cost_hamiltonian.generate_instance(number_of_qubits=number_of_qubits,
                                                                seed=seed_cost_hamiltonian,
                                                                read_from_drive_if_present=True)

print("Class description (cost):", cost_hamiltonian.hamiltonian_class_description)
print("Instance description (cost):", cost_hamiltonian.hamiltonian_instance_description)

# if we wish, we can solve the Hamiltonian
if cost_hamiltonian.lowest_energy is None:
    cost_hamiltonian.solve_hamiltonian(both_directions=True)

ground_state_energy = cost_hamiltonian.ground_state_energy
highest_energy = cost_hamiltonian.highest_energy

cost_hamiltonian.hamiltonian_class_specifier.get_description_string()



File not found!
FILE NOT FOUND!
Class description (cost): HMN=SK;LOC=(2,);CFD=CT~DIS_CDN~UNI_CDP~low~-1_high~1_step~1
Instance description (cost): NOQ=21;HII=1


'HMN=SK;LOC=(2,);CFD=CT~DIS_CDN~UNI_CDP~low~-1_high~1_step~1'

### Noise-Directed Adaptive Remapping

* We will now run NDAR (see Ref.~[1]) in a loop with the optimizer we used in the previous notebook.
* We will use stronger readout noise model and fewer samples, because we simulate only small system, so it's pretty easy to find ground state even from suboptimal points with noise.
* We will use small number of function calls and samples for the same reason.
* Again, this noise is not realistic, but it is sufficient to demonstrate the NDAR algorithm.

In [7]:
from quapopt.optimization.QAOA.implementation.QAOARunnerSampler import QAOARunnerSampler
from quapopt.circuits.noise.simulation.ClassicalMeasurementNoiseSampler import \
    ClassicalMeasurementNoiseSampler, MeasurementNoiseType
from quapopt.optimization.parameter_setting.variational.scipy_tools.ScipyOptimizerWrapped import ScipyOptimizerWrapped

#Number of objective function calls
number_of_function_calls = 30
#Number of measurements to estimate the expectation values
number_of_samples = 75

#We can specify the QAOA depth here.
qaoa_depth = 1

#we can specify the details of classical optimizer here.
classical_optimizer = ScipyOptimizerWrapped(parameters_bounds=[(-np.pi, np.pi)] * 2 * int(qaoa_depth),
                                            optimizer_name='COBYQA',
                                            optimizer_kwargs=None,
                                            basinhopping=True,
                                            basinhopping_kwargs={'niter': 3},
                                            starting_point=[0.05] * 2 * int(qaoa_depth)
                                            )

# Fully asymmetric noise -- equivalent to amplitude damping at the end of the circuit
p_01 = 1 - 1 / number_of_qubits
p_10 = None

In [8]:
from quapopt.meta_algorithms.NDAR.NDARRunnerQAOA import NDARRunnerQAOA
from quapopt.meta_algorithms.NDAR import (ConvergenceCriterionNames,
                                          ConvergenceCriterion)

seed_main = 1

#Specify the convergence criterion for NDAR
#Here we use the maximum number of unsuccessful trials.
#If after 5 iterations the NDAR does not find a better solution, it stops.
max_iterations = 5
ndar_convergence_criterion = ConvergenceCriterion(
    convergence_criterion_name=ConvergenceCriterionNames.MaxUnsuccessfulTrials,
    convergence_value=max_iterations)
#Here we specify the sampler class and its arguments
#It is abstract because we sometimes use it with different samplers (not QAOA)
qaoa_sampler_class = QAOARunnerSampler

#Heuristic parameter for potentially re-gauging towards energy-increasing states.
#Value 0.0 means always gauge-transforming.
temperature = 0.0

experiment_set_id = f"{anf.create_random_uuid()}"

print("Experiment set id:", experiment_set_id)

#Here we specify the logger arguments
logger_kwargs_ndar = {'experiment_folders_hierarchy':
                          ['SimulationResults',
                           'TestingNDAR'],
                      'experiment_set_id': experiment_set_id,
                      }

logging_level = LoggingLevel.DETAILED


Experiment set id: bc7faa98308344ce94fa5b0c121873b0


In [9]:
numpy_rng_sampling = np.random.default_rng(seed=seed_main)
numpy_rng_boltzmann = np.random.default_rng(seed=seed_main)
numpy_rng_noise = np.random.default_rng(seed=seed_main)
qaoa_sampler_kwargs = {'numpy_rng_sampling': numpy_rng_sampling}

#Set up the classical measurement noise sampler
CMNS = ClassicalMeasurementNoiseSampler(noise_type=MeasurementNoiseType.TP_1q_identical,
                                        noise_description={'p_01': p_01,
                                                           'p_10': p_10},
                                        rng=numpy_rng_noise)

input_hamiltonian_representations = [cost_hamiltonian.copy()]

# Create the NDAR runner instance.
ndar_runner = NDARRunnerQAOA(input_hamiltonian_representations=input_hamiltonian_representations,
                             convergence_criterion=ndar_convergence_criterion,
                             sampler_class=qaoa_sampler_class,
                             attractor_model=None,
                             logging_level=logging_level,
                             logger_kwargs=logger_kwargs_ndar
                             )
print("GROUNDS STATE ENERGY:", ground_state_energy)

print("Experiment set id:", experiment_set_id)

#Note: For the sake of reducing simulation complexity when known solutions are known, the runner will actually break when reaching ground state.
best_result_ndar, optimization_history_ndar = ndar_runner.run_NDAR(
    #QAOA kwargs
    qaoa_depth=qaoa_depth,
    number_of_function_calls=number_of_function_calls,
    classical_optimizer=classical_optimizer,
    number_of_samples_per_function_call=number_of_samples,
    measurement_noise=CMNS,
    numpy_rng_sampling=numpy_rng_sampling,
    ##### NDAR KWARGS
    numpy_rng_boltzmann=numpy_rng_boltzmann,
    #Generate optimizer seed for each optimization in a loop (argument is iteration index)
    step_seed_generator=lambda x: x,
    show_progress_bar_ndar=True,
    temperature_NDAR=temperature
)



No existing metadata found for the specified experiment set. 
GROUNDS STATE ENERGY: -70.0
Experiment set id: bc7faa98308344ce94fa5b0c121873b0


  0%|          | 0/1000 [00:00<?, ?it/s]

Iteration:  0
Best energy so far:  -18.0 (AR: 0.6176)
Iteration:  1
Best energy so far:  -38.0 (AR: 0.7647)
Iteration:  2
Best energy so far:  -50.0 (AR: 0.8529)
Iteration:  3
Best energy so far:  -54.0 (AR: 0.8824)
Iteration:  4
Best energy so far:  -62.0 (AR: 0.9412)
Iteration:  5
Best energy so far:  -66.0 (AR: 0.9706)
Iteration:  6
Best energy so far:  -70.0 (AR: 1.0)
FOUND GROUND STATE! breaking

Finished after  7 iterations.
Final best energy: np.float64(-70.0)
Optimization time: 59.6619484369985


In [10]:
from quapopt.optimization.parameter_setting.variational.QAOAOptimizationRunner import QAOAOptimizationRunner
#Here we can check if noiseless QAOA with the same number of samples and function calls would find better candidate solution
qaoa_runner_sampler = qaoa_sampler_class(hamiltonian_representations_cost=[cost_hamiltonian.copy()],
                                         hamiltonian_representations_phase=None,
                                         **qaoa_sampler_kwargs)
qaoa_runner_sampler.initialize_backend_python(backend='auto')

qaoa_optimizer = QAOAOptimizationRunner(qaoa_runner=qaoa_runner_sampler)

best_n_results_noiseless_qaoa, optimization_result_noiseless_qaoa = qaoa_optimizer.run_optimization(
    qaoa_depth=qaoa_depth,
    number_of_function_calls=number_of_function_calls * len(
        optimization_history_ndar),
    classical_optimizer=classical_optimizer,
    optimizer_seed=1,
    number_of_samples=number_of_samples,
    numpy_rng_sampling=numpy_rng_sampling,
    show_progress_bar=True,
    measurement_noise=None,
    )
anf.cool_print("BEST ENERGY OF NOISELESS QAOA:", best_n_results_noiseless_qaoa[0][0], 'blue')
anf.cool_print("BEST ENERGY OF NDAR:", best_result_ndar[0], 'red')

COBYQA:   0%|          | 0/210 [00:00<?, ?it/s]

BEST ENERGY OF NOISELESS QAOA: np.float64(-70.0)
BEST ENERGY OF NDAR: np.float64(-70.0)
NDAR better than noiseless QAOA: False


### Reading data

* In a moment, we will want to visualize data.
* We could take the data directly from the output of optimization, but here we will read it from database to test the logging:
* To this aim, we use a helper function imported from NDAR submodule

In [11]:
from quapopt.data_analysis.data_handling import (STANDARD_NAMES_DATA_TYPES as SNDT,
                                                 STANDARD_NAMES_VARIABLES as SNV, )
from quapopt.data_analysis.data_handling import ResultsLogger

results_logger = ResultsLogger(**logger_kwargs_ndar)
ndar_data = results_logger.gather_results(data_type=SNDT.NDAROverview, )
ndar_data

,NDARIteration,Bitflip,AttractorModel,ConvergenceCriterion,TrialIndex,HamiltonianRepresentationIndex,Angles,EnergyMean,EnergyBest,BitstringBest,ExperimentInstanceID,SourceFileName,SourceFilePath
0,0,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,3,0,"[-0.95, 0.05]",8.826667,-18.0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...",a3d34fb329d94134aba4c7f06b644e74,"ESI=bc7faa98308344ce94fa5b0c121873b0,dat=ndo.csv",/home/fipeczek/python_packages/global-data-sto...
1,1,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,11,0,"[-1.0210345822389755, 0.42441596303543744]",-17.200000,-38.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",9b2de7da58744fa2b46fbfe5cf931720,"ESI=bc7faa98308344ce94fa5b0c121873b0,dat=ndo.csv",/home/fipeczek/python_packages/global-data-sto...
2,2,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,37,0,"[1.6737528266992288, -0.7660550345125372]",-33.680000,-50.0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...",d2a2530b336c4a5fba43d46357fa807f,"ESI=bc7faa98308344ce94fa5b0c121873b0,dat=ndo.csv",/home/fipeczek/python_packages/global-data-sto...
3,3,"(np.int64(1), np.int64(1), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,18,0,"[2.4346162465117933, -1.1966710152909257]",-45.360000,-54.0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1702f7c64c634053b89028b52056ab47,"ESI=bc7faa98308344ce94fa5b0c121873b0,dat=ndo.csv",/home/fipeczek/python_packages/global-data-sto...
4,4,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,35,0,"[2.9762803245005744, -0.5802713331355986]",-47.493333,-62.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",cfb2b03e97584c12b4a6556a3a288185,"ESI=bc7faa98308344ce94fa5b0c121873b0,dat=ndo.csv",/home/fipeczek/python_packages/global-data-sto...
5,5,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,5,0,"[0.023921915419443475, -1.9496599089213338]",-56.293333,-66.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",bcb94e5975e345429c869a9dccc5f53c,"ESI=bc7faa98308344ce94fa5b0c121873b0,dat=ndo.csv",/home/fipeczek/python_packages/global-data-sto...
6,6,"(np.int64(0), np.int64(0), np.int64(0), np.int...",ATS=0,COC~ConvergenceCriterionNames.MaxUnsuccessfulT...,5,0,"[-0.31301369863016093, -0.10000000000001101]",-60.240000,-70.0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ...",11474450615748c1a387dcca4ec5cba8,"ESI=bc7faa98308344ce94fa5b0c121873b0,dat=ndo.csv",/home/fipeczek/python_packages/global-data-sto...


### Visualization

* Let's take a look at a simple visualization of the NDAR optimization.

In [12]:
import plotly
import plotly.graph_objects as go

plotly.io.templates.default = "plotly"
plotly.offline.init_notebook_mode(connected=True)

In [14]:
xs = ndar_data[SNV.NDARIteration.id_long].tolist()
ys = ndar_data[SNV.EnergyBest.id_long].tolist()
#convert to approximation ratios
ys = [(highest_energy - Ei) / (highest_energy - ground_state_energy) for Ei in ys]
ndar_figure = go.Figure()
ndar_figure.add_trace(go.Scatter(x=xs, y=ys, mode='lines+markers', name=f'NDAR p(0|1) = {p_01}'))

ndar_figure.update_layout(
    title=f"NDAR (T = {temperature}) Performance for {number_of_qubits} qubits on {cost_hamiltonian.hamiltonian_class_specifier.get_description_string()}",
    xaxis_title="NDAR Iteration",
    yaxis_title="Approximation Ratio",
    template="plotly",
    font=dict(size=14),
    hovermode="x unified"
)
ndar_figure.update_yaxes(range=[0.5, 1.03])
#update xticks to only NDAR iterations:
ndar_figure.update_xaxes(tickvals=xs)

plotly.offline.plot(ndar_figure,
                    filename=f'../temp/NDAR_performance.html')
ndar_figure.show()